In [ ]:
import sys
import os
sys.path.insert(0, os.path.abspath('../'))

import plotly.plotly as py
import plotly.graph_objs as go

from plotly.offline import init_notebook_mode, iplot

# Import stuff for making the plot interactive
from ipywidgets import interactive, HBox, VBox, widgets, Label, Layout
import numpy as np

init_notebook_mode(connected=True)

np.set_printoptions(precision=6)
np.set_printoptions(suppress=True)

In [ ]:
import trjgen.trjgen_core as tj
import trjgen.trjgen_helpers as tjh
import trjgen.pltly_helpers as ply
import trjgen.class_pwpoly as pw

In [ ]:
# Helper functions

def ToQuaternion(yaw, pitch, roll):
    q = np.zeros((4))
    
    cy = np.cos(yaw * 0.5)
    sy = np.sin(yaw * 0.5)
    cp = np.cos(pitch * 0.5)
    sp = np.sin(pitch * 0.5)
    cr = np.cos(roll * 0.5)
    sr = np.sin(roll * 0.5)

    q[0] = cy * cp * cr + sy * sp * sr
    q[1] = cy * cp * sr - sy * sp * cr
    q[2] = sy * cp * sr + cy * sp * cr
    q[3] = sy * cp * cr - cy * sp * sr
    
    return q;

# Generate the matrices for the interpolation problem
def getInterpolMatrices(tg, tg_q, v_norm, a_norm, Trec):
    
    # Extract the coordinates of the target Z axis from the rotation matrix
    # extressed with the quaternion
    tg_Zi = np.array([2.0*tg_q[0]*tg_q[2] + 2.0*tg_q[1]*tg_q[3],
         2.0*(tg_q[2]*tg_q[3] - tg_q[0]*tg_q[1]),
         tg_q[0]*tg_q[0] -tg_q[1]*tg_q[1] -tg_q[2]*tg_q[2] + tg_q[3]*tg_q[3]]) 

    a_dem = a_norm * tg_Zi - np.array([0.0, 0.0, 9.81])
    v_dem = - v_norm * tg_Zi
    
    # Compute the waypoints near the target
    p_end = tg + v_dem * Trec
    

    print("Target = \n", tg)
    print("Vel = \n", v_dem)
    print("Acc = \n", a_dem)
    print("Recoil point = \n ", p_end)

    #   Relative waypoint data
    #   Start   PreTarget   Target      PostTarget  End
    X = np.array([
        [ 0,   tg[0],     p_end[0]],
        [ 0,   v_dem[0],  0.0],
        [ 0,   a_dem[0],  0.0],
        [ 0,   np.nan,    0.0]
    ])

    Y = np.array([
        [ 0,  tg[1],    p_end[1]],
        [ 0,  v_dem[1], 0.0],
        [ 0,  a_dem[1], 0.0],
        [ 0,  np.nan,   0.0]
    ])

    Z = np.array([
        [ 0,  tg[2],     p_end[2]],
        [ 0,  v_dem[2],  0.0],
        [ 0,  a_dem[2],  0.0],
        [ 0,  np.nan,    0.0]
    ])

    W = np.array([
        [ 0,   np.nan,     0.0],
        [ 0,   np.nan,     0.0],
        [ 0,   np.nan,     0.0],
        [ 0,   np.nan,     0.0]
    ])
    
    return (X, Y, Z, W)

# Create the knots vector
def updateKnots(t_impact, Trec):
    t2 = t_impact + Trec
        
    knots = np.array([0,
                      t_impact,
                      t2]
                    )
    return knots
    


In [ ]:
## Evaluate initial data
vehicle_mass = 0.032
thrust_thr = 9.81 * vehicle_mass * 2.0

t_impact = 1.8
Trec = 10

# Polynomial characteristic:  order
ndeg = 7
print('Test with {:d}th order polynomial.'.format(ndeg))
## Waipoints in the flat output space (or dimension 3)
nconstr = 4

# Dummy target data for initial data generation
Xt = -2.4
Yt = -0.5
Zt = 0.1

roll_t = 0.0
pitch_t = np.pi/2
yaw_t = 0.0

Vt = 1.5
At = 9.0

a_norm = At
v_norm = Vt

# Target Position
tg_rel = np.array([Xt, Yt, Zt])
tg_q = ToQuaternion(roll_t, pitch_t, yaw_t)

# Generate the initial interpolation matrices
(X, Y, Z, W) = getInterpolMatrices(tg_rel, tg_q, v_norm, a_norm, Trec)

In [ ]:
## Polynomial generation

# Knots vector
knots = updateKnots(t_impact, Trec)

ppx = pw.PwPoly(X, knots, ndeg)
ppy = pw.PwPoly(Y, knots, ndeg)
ppz = pw.PwPoly(Z, knots, ndeg)
ppw = pw.PwPoly(W, knots, ndeg)


## Plotting the initial trajectory
tv = np.linspace(0, max(knots), 100);
    
# Generate the first trajectory
(Xtj, Ytj, Ztj, Wtj, Zbtj) = tjh.TrajFromPW(tv, [0,1,2], \
    pwpolx=ppx, pwpoly=ppy, pwpolz=ppz, pwpolw = ppw)

# Compute the thrust limits
(ffthrust, available_thrust) = tjh.getlimits(tv, Xtj, Ytj, Ztj, \
    vehicle_mass, thrust_thr)

perc_available = (available_thrust / thrust_thr) * 100.0


In [ ]:
# Define initial TRACES and DATA
#
# Figure {
#   Data
#   Layout
#}
#
# Data {
#   Trace
#   Type
#}
#
# The Trace describes the kind of representation and the data to be represented.
#

# Defined as an object
trace1 = go.Scatter3d(
            x = Xtj[0, :],
            y = Ytj[0, :],
            z = Ztj[0, :],
            mode = 'markers',
            marker = dict(
                size = 4,
                color = tv,
                colorscale = 'Viridis',
                opacity = 0.8,
                colorbar = dict (
                    thickness = 20,
                    len =  0.5,
                    x = 0.8,
                    y = 0.6
                    )
                ),
            name = '3D Path'
            )

# Defined as an object
trace2 = {
            "type": "cone",
            "x": Xtj[0,:],
            "y": Ytj[0,:],
            "z": Ztj[0,:],
            "u": Zbtj[0,:],
            "v": Zbtj[1,:],
            "w": Zbtj[2,:],
            "customdata": tv,
            "sizemode": "absolute",
            "colorscale": 'Blues',
            "hoverinfo": "u+v+w",
            "sizeref": 10000,
            "visible": False,
            "showscale": False
    }

trace3 = go.Scatter3d(
        x = Xtj[0, :],
        y = Ytj[0, :],
        z = Ztj[0, :],
        mode = 'markers',
        marker = dict(
            size = 4,
            color = perc_available,
            colorscale = 'Viridis',
            opacity = 0.8,
            colorbar = dict ( thickness = 20, len =  0.5, 
                             x = 0.8, y = 0.6)
            ),
        name = '3D Path with thrust free margin',
        visible = False
        )


## ==================================
## Data is a list of traces.
data = [trace1, trace2, trace3]

In [ ]:
## Interactive element and associated methods
#
# 1) Slider to select the intercept time
# 2) Box to select the recovery

# 1) Plot selector Widget
# It is possible to choose what to visualize among Trajectory time, Attitude and Thrust limit
plotSelector_w = widgets.Dropdown(
    options=['Time', 'Attitude', 'Available Thrust'],
    value='Time',
    description='Plot Type',
)

# 2) Box for the time recovery time Widget
boxFTime_w = widgets.FloatText(
    value = Trec,
    step = 0.25,
    description = "Recovery Time"
)

# 3) Boxes for final point
boxFXPoint_w = widgets.FloatText(
    value = tg_rel[0],
    step = 0.05,
    description = 'Xf'
)
boxFYPoint_w = widgets.FloatText(
    value = tg_rel[1],
    step = 0.05,
    description = 'Yf'
)
boxFZPoint_w = widgets.FloatText(
    value = tg_rel[2],
    step = 0.05,
    description = 'Zf'
)

# 3) Slider for intercept time Widget
#    The slider goes from 0 to 5.0 secs
sliderITime_w = widgets.FloatSlider(min=0.0, max=5.0, step=0.1, value=t_impact, description = "Intercept Time", width=200)





#####
boxFileName_w = widgets.Text(
    value='poly.csv',
    placeholder='Filename',
    description='Output File:',
    disabled=False, width=200
)




## ==========================================================================================================
## CALLBACKS

# This callback modify the Stop time after the target
def finalTime_response(change):
    global ppx, ppy, ppz, ppw
    global X, Y, Z, W
    global knots
    
    # Update knots to consider the new Recovery time
    # The impact time is taken from the slider, whilst the Recover Time is taken 
    # from the textbox
    knots = updateKnots(sliderITime_w.value, boxFTime_w.value)
    
    # Recompute the trajectory
    tv = np.linspace(0, max(knots), 100)
    
    ppx = pw.PwPoly(X, knots, ndeg)
    ppy = pw.PwPoly(Y, knots, ndeg)
    ppz = pw.PwPoly(Z, knots, ndeg)
    ppw = pw.PwPoly(W, knots, ndeg)
    
    (Xtj, Ytj, Ztj, Wtj, Zbtj) = tjh.TrajFromPW(tv, [0,1,2], \
        pwpolx=ppx, pwpoly=ppy, pwpolz=ppz, pwpolw=ppw)
    
    (ffthrust, available_thrust) = tjh.getlimits(tv, Xtj, Ytj, Ztj, \
        vehicle_mass, thrust_thr)
    perc_available = (available_thrust / thrust_thr) * 100.0

    # Update the data in the traces
    
    figure_w.data[0].x = Xtj[0, :]
    figure_w.data[0].y = Ytj[0, :]
    figure_w.data[0].z = Ztj[0, :]
    figure_w.data[0].marker.color = tv
    
    figure_w.data[1].x = Xtj[0, :]
    figure_w.data[1].y = Ytj[0, :]
    figure_w.data[1].z = Ztj[0, :]
    figure_w.data[1].u = Zbtj[0, :]
    figure_w.data[1].v = Zbtj[1, :]
    figure_w.data[1].w = Zbtj[2, :]
        
    figure_w.data[2].x = Xtj[0, :]
    figure_w.data[2].y = Ytj[0, :]
    figure_w.data[2].z = Ztj[0, :]
    figure_w.data[2].marker.color = perc_available


def plotSelector_response(change):
#    with figure_w.batch_update():
    if (plotSelector_w.value == 'Time'):
        figure_w.data[0].visible = True
        figure_w.data[1].visible = False
        figure_w.data[2].visible = False
    if (plotSelector_w.value == 'Attitude'):
        figure_w.data[0].visible = False
        figure_w.data[1].visible = True
        figure_w.data[2].visible = False
    if (plotSelector_w.value == 'Available Thrust'):
        figure_w.data[0].visible = False
        figure_w.data[1].visible = False
        figure_w.data[2].visible = True



def update_impactTime(change):
    global ppx, ppy, ppz, ppw
    global X, Y, Z, W
    global knots
    
    # Update knots given the impact time
    knots = updateKnots(sliderITime_w.value, boxFTime_w.value)
    tv = np.linspace(0,max(knots),100);
    
    ppx = pw.PwPoly(X, knots, ndeg)
    ppy = pw.PwPoly(Y, knots, ndeg)
    ppz = pw.PwPoly(Z, knots, ndeg)
    ppw = pw.PwPoly(W, knots, ndeg)
    
    (Xtj, Ytj, Ztj, Wtj, Zbtj) = tjh.TrajFromPW(tv, [0,1,2], \
        pwpolx=ppx, pwpoly=ppy, pwpolz=ppz, pwpolw = ppw)
    
    (ffthrust, available_thrust) = tjh.getlimits(tv, Xtj, Ytj, Ztj, \
        vehicle_mass, thrust_thr)
    perc_available = (available_thrust / thrust_thr) * 100.0

    figure_w.data[0].x = Xtj[0, :]
    figure_w.data[0].y = Ytj[0, :]
    figure_w.data[0].z = Ztj[0, :]
    figure_w.data[0].marker.color = tv
    
    figure_w.data[1].x = Xtj[0, :]
    figure_w.data[1].y = Ytj[0, :]
    figure_w.data[1].z = Ztj[0, :]
    figure_w.data[1].u = Zbtj[0, :]
    figure_w.data[1].v = Zbtj[1, :]
    figure_w.data[1].w = Zbtj[2, :]
        
    figure_w.data[2].x = Xtj[0, :]
    figure_w.data[2].y = Ytj[0, :]
    figure_w.data[2].z = Ztj[0, :]
    figure_w.data[2].marker.color = perc_available


def Xf_response(change):
    global ppx, ppy, ppz, ppw
    global X, Y, Z, W
    global tg_rel
    
    tg_rel[0] = boxFXPoint_w.value
    (X, Y, Z, W) = getInterpolMatrices(tg_rel, tg_q, v_norm, a_norm, boxFTime_w.value)
    
    #tv = np.linspace(0,max(knots),100);
    
    ppx = pw.PwPoly(X, knots, ndeg)
    ppy = pw.PwPoly(Y, knots, ndeg)
    ppz = pw.PwPoly(Z, knots, ndeg)
    ppw = pw.PwPoly(W, knots, ndeg)
    
    (Xtj, Ytj, Ztj, Wtj, Zbtj) = tjh.TrajFromPW(tv, [0,1,2], \
        pwpolx=ppx, pwpoly=ppy, pwpolz=ppz, pwpolw = ppw)
    
    (ffthrust, available_thrust) = tjh.getlimits(tv, Xtj, Ytj, Ztj, \
        vehicle_mass, thrust_thr)
    perc_available = (available_thrust / thrust_thr) * 100.0

    figure_w.data[0].x = Xtj[0, :]
    figure_w.data[0].y = Ytj[0, :]
    figure_w.data[0].z = Ztj[0, :]
    figure_w.data[0].marker.color = tv
    
    figure_w.data[1].x = Xtj[0, :]
    figure_w.data[1].y = Ytj[0, :]
    figure_w.data[1].z = Ztj[0, :]
    figure_w.data[1].u = Zbtj[0, :]
    figure_w.data[1].v = Zbtj[1, :]
    figure_w.data[1].w = Zbtj[2, :]
        
    figure_w.data[2].x = Xtj[0, :]
    figure_w.data[2].y = Ytj[0, :]
    figure_w.data[2].z = Ztj[0, :]
    figure_w.data[2].marker.color = perc_available
    
def Yf_response(change):
    global ppx, ppy, ppz, ppw
    global X, Y, Z, W
    global tg_rel

    tg_rel[1] = boxFYPoint_w.value
    (X, Y, Z, W) = getInterpolMatrices(tg_rel, tg_q, v_norm, a_norm, boxFTime_w.value)
    
    #tv = np.linspace(0,max(knots),100);
    
    ppx = pw.PwPoly(X, knots, ndeg)
    ppy = pw.PwPoly(Y, knots, ndeg)
    ppz = pw.PwPoly(Z, knots, ndeg)
    ppw = pw.PwPoly(W, knots, ndeg)
    
    (Xtj, Ytj, Ztj, Wtj, Zbtj) = tjh.TrajFromPW(tv, [0,1,2], \
        pwpolx=ppx, pwpoly=ppy, pwpolz=ppz, pwpolw = ppw)
    
    (ffthrust, available_thrust) = tjh.getlimits(tv, Xtj, Ytj, Ztj, \
        vehicle_mass, thrust_thr)
    perc_available = (available_thrust / thrust_thr) * 100.0

    figure_w.data[0].x = Xtj[0, :]
    figure_w.data[0].y = Ytj[0, :]
    figure_w.data[0].z = Ztj[0, :]
    figure_w.data[0].marker.color = tv
    
    figure_w.data[1].x = Xtj[0, :]
    figure_w.data[1].y = Ytj[0, :]
    figure_w.data[1].z = Ztj[0, :]
    figure_w.data[1].u = Zbtj[0, :]
    figure_w.data[1].v = Zbtj[1, :]
    figure_w.data[1].w = Zbtj[2, :]
        
    figure_w.data[2].x = Xtj[0, :]
    figure_w.data[2].y = Ytj[0, :]
    figure_w.data[2].z = Ztj[0, :]
    figure_w.data[2].marker.color = perc_available
    
def Zf_response(change):
    global ppx, ppy, ppz, ppw
    global X, Y, Z, W
    global tg_rel

    tg_rel[2] = boxFZPoint_w.value
    (X, Y, Z, W) = getInterpolMatrices(tg_rel, tg_q, v_norm, a_norm, boxFTime_w.value)
    
    ppx = pw.PwPoly(X, knots, ndeg)
    ppy = pw.PwPoly(Y, knots, ndeg)
    ppz = pw.PwPoly(Z, knots, ndeg)
    ppw = pw.PwPoly(W, knots, ndeg)
    
    (Xtj, Ytj, Ztj, Wtj, Zbtj) = tjh.TrajFromPW(tv, [0,1,2], \
        pwpolx=ppx, pwpoly=ppy, pwpolz=ppz, pwpolw = ppw)
    
    (ffthrust, available_thrust) = tjh.getlimits(tv, Xtj, Ytj, Ztj, \
        vehicle_mass, thrust_thr)
    perc_available = (available_thrust / thrust_thr) * 100.0
    
    figure_w.data[0].x = Xtj[0, :]
    figure_w.data[0].y = Ytj[0, :]
    figure_w.data[0].z = Ztj[0, :]
    figure_w.data[0].marker.color = tv
    
    figure_w.data[1].x = Xtj[0, :]
    figure_w.data[1].y = Ytj[0, :]
    figure_w.data[1].z = Ztj[0, :]
    figure_w.data[1].u = Zbtj[0, :]
    figure_w.data[1].v = Zbtj[1, :]
    figure_w.data[1].w = Zbtj[2, :]
        
    figure_w.data[2].x = Xtj[0, :]
    figure_w.data[2].y = Ytj[0, :]
    figure_w.data[2].z = Ztj[0, :]
    figure_w.data[2].marker.color = perc_available
    
# Now I have to link the widgets to callbacks
boxFTime_w.observe(finalTime_response, names = 'value')
plotSelector_w.observe(plotSelector_response, names="value")
sliderITime_w.observe(update_impactTime, names='value')

boxFXPoint_w.observe(Xf_response, names = 'value')
boxFYPoint_w.observe(Yf_response, names = 'value')
boxFZPoint_w.observe(Zf_response, names = 'value')

In [ ]:
## Define Plot LAYOUT

mylayout = go.Layout(
    autosize=True,
    width=900,
    height=800,
    margin = dict( l = 0, r = 0, b = 0, t = 0),
    scene={"aspectmode": "data",
                "xaxis": {"title": f"x [m]"},
                "yaxis": {"title": f"y [m]"},
                "zaxis": {"title": f"z [m]"}
           },
    xaxis=go.layout.XAxis(
        automargin=True
    ),
    yaxis=go.layout.YAxis(
        automargin=True
    ),
)

figure = go.Figure(data=data, layout=mylayout)

# Interactive elements
figure_w = go.FigureWidget(figure)

# Button to generate trajectory file
button1 = widgets.Button(description="Generate Info")
button2 = widgets.Button(description="Save Polynomial")


def on_button_INFO_clicked(b):
    outputfile = "trajectory_info.txt"
    print(" =======  Trajectory Info ======")
    def writeMat(f, X):
        for i in range(X.shape[0]):
            for j in range(X.shape[1]):
                f.write('{:5f}, '.format(X[i,j]))
                str = "{:5f}, ".format(X[i,j])
                print(str, end="")
            f.write('\n')
            print("")

    f = open(outputfile, "w")
    f.write("\nKnots:\n")
    print("Knots:")
    for i in range(knots.size):
        f.write('{:5f}, '.format(knots[i]))
        str = "{:5f}, ".format(knots[i])
        print(str, end="")
    f.write('\n')
    print("");
    
    f.write("\nData X:\n")
    print("\nData X: ")
    writeMat(f, X)
    
    f.write("\nData Y\n")
    print("Data Y: ")
    writeMat(f, Y)

    f.write("\nData Z\n")
    print("Data Z: ")
    writeMat(f, Z)

    f.close()

def on_button_SAVE_clicked(b):
    global ppx, ppy, ppz, ppw
    print("Saving polynomial file...")
    # Save the polynomial coefficients on file
    x_coeff = ppx.getCoeffMat();
    y_coeff = ppy.getCoeffMat();
    z_coeff = ppz.getCoeffMat();
    w_coeff = ppy.getCoeffMat();

    Dt = knots[1:len(knots)] - knots[0:len(knots)-1]
    tj.pp2file(Dt, x_coeff, y_coeff, z_coeff, w_coeff, boxFileName_w.value)
    
    # Check out the saved file
    file = open(boxFileName_w.value, 'r')
    print(file.read())

button1.on_click(on_button_INFO_clicked)
button2.on_click(on_button_SAVE_clicked)


#######################################################################
# MENU CONTAINERS
label_layout = Layout(width='600px',height='50px')
menu1_container = widgets.HBox([plotSelector_w, boxFTime_w], layout = label_layout)
menu2_container = widgets.HBox([sliderITime_w])
menu3_container = widgets.VBox([boxFXPoint_w, boxFYPoint_w, boxFZPoint_w])
menu4_container = widgets.HBox([boxFileName_w, button1, button2])


#######################################################################    
# display the FigureWidget and slider with center justification
vb = VBox((menu1_container, menu2_container, menu3_container, figure_w, menu4_container))
vb.layout.align_items = 'center'


In [ ]:
vb